# Creating matrices for pairs of labels

## Which ones?

- Sequence-based ML model F1 scores
- Sequence similarity scores
- annotation similarity scores
- annotation overlap

## For which dataset?

- First for yeast because of conference

In [1]:
from subpred.util import load_df
import networkx as nx
import pandas as pd
from subpred.transmembrane_transporters import get_transmembrane_transporter_dataset

In [2]:
# TODO TransporterData class, with these parameters and the dataframes as fields

In [3]:
ORGANISM_IDS={559292}
SWISSPROT_ONLY=False
MAX_SEQUENCE_EVIDENCE_CODE = 1
EXCLUDE_IEA_GO_TERMS=False

In [4]:
df_sequences, df_uniprot_goa, df_go_chebi = get_transmembrane_transporter_dataset(
    organism_ids=ORGANISM_IDS,
    swissprot_only=SWISSPROT_ONLY,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=EXCLUDE_IEA_GO_TERMS,
    max_sequence_evidence_code=MAX_SEQUENCE_EVIDENCE_CODE,
)
display(df_sequences)
display(df_uniprot_goa)
display(df_go_chebi)

,sequence,reviewed,protein_existence,organism_id,protein_names
Uniprot,,,,,
P00401,MVQRWLYSTNAKDIAVLYFMLAIFSGMAGTAMSLIIRLELAAPGSQ...,True,1,559292,Cytochrome c oxidase subunit 1 (EC 7.1.1.9) (C...
P00830,MVLPRLYTATSRAAFKAAKQSAPLLSTSWKRCMASAAQSTPITGKV...,True,1,559292,"ATP synthase subunit beta, mitochondrial (EC 7..."
P04817,MTNSKEDADIEEKHMYNEPVTTLFHDVEASQTHHRRGSIPLKDEKS...,True,1,559292,Arginine permease CAN1 (Canavanine resistance ...
P07213,MKSFITRNKTAILATVAATGTAIGAYYYYNQLQQQQQRGKKNTINK...,True,1,559292,Mitochondrial import receptor subunit TOM70 (7...
P14906,MPTNYEYDEASETWPSFILTGLLMVVGPMTLLQIYQIFFGANAEDG...,True,1,559292,Protein translocation protein SEC63 (Protein N...
...,...,...,...,...,...
P39542,MFQQLSASIRHNAHIIFLCISWYFISSLASQVTKQVLTVCPLPLFL...,True,1,559292,Uncharacterized transporter YJL193W
Q05497,MAGILSKTLSEVHPSLRTNGMGIGNTHRRISLGFLPPNKKNPLVRK...,True,1,559292,Uncharacterized transporter YDR338C
P38318,MEPKRKSGSLAKHDLPQFYLLIMLYLAQGIPVGLAFGTVPFLLKSL...,True,1,559292,Uncharacterized membrane protein YBR220C


,Uniprot,qualifier,go_id,go_term,evidence_code,aspect,go_id_ancestor,go_term_ancestor
0,D6W196,enables,GO:0005347,ATP transmembrane transporter activity,IBA,F,GO:0015215,nucleotide transmembrane transporter activity
1,D6W196,enables,GO:0005347,ATP transmembrane transporter activity,IBA,F,GO:0005347,ATP transmembrane transporter activity
2,D6W196,enables,GO:0005347,ATP transmembrane transporter activity,IBA,F,GO:1901505,carbohydrate derivative transmembrane transpor...
3,D6W196,enables,GO:0005347,ATP transmembrane transporter activity,IBA,F,GO:0008514,organic anion transmembrane transporter activity
4,D6W196,enables,GO:0005347,ATP transmembrane transporter activity,IBA,F,GO:0022857,transmembrane transporter activity
...,...,...,...,...,...,...,...,...
7732,Q9ZZX1,enables,GO:0004129,cytochrome-c oxidase activity,IEA,F,GO:0008324,monoatomic cation transmembrane transporter ac...
7733,Q9ZZX1,enables,GO:0004129,cytochrome-c oxidase activity,IEA,F,GO:0015075,monoatomic ion transmembrane transporter activity
7734,Q9ZZX1,enables,GO:0004129,cytochrome-c oxidase activity,IEA,F,GO:0015453,oxidoreduction-driven active transmembrane tra...
7735,Q9ZZX1,enables,GO:0004129,cytochrome-c oxidase activity,IEA,F,GO:0022804,active transmembrane transporter activity


,go_id,go_term,chebi_id,chebi_term,chebi_go_relation
0,GO:0000064,L-ornithine transmembrane transporter activity,CHEBI:46911,L-ornithinium(1+),has_primary_input
1,GO:0000095,S-adenosyl-L-methionine transmembrane transpor...,CHEBI:59789,S-adenosyl-L-methionine zwitterion,has_primary_input
2,GO:0000099,sulfur amino acid transmembrane transporter ac...,CHEBI:26834,sulfur-containing amino acid,has_primary_input
3,GO:0000100,S-methylmethionine transmembrane transporter a...,CHEBI:58252,S-methyl-L-methionine zwitterion,has_primary_input
4,GO:0000102,L-methionine secondary active transmembrane tr...,CHEBI:57844,L-methionine zwitterion,has_primary_input
...,...,...,...,...,...
379,GO:1901682,sulfur compound transmembrane transporter acti...,CHEBI:26835,sulfur molecular entity,has_primary_input
380,GO:1902557,5'-adenylyl sulfate transmembrane transporter ...,CHEBI:58243,5'-adenylyl sulfate(2-),has_primary_input
381,GO:1903089,5-amino-1-ribofuranosylimidazole-4-carboxamide...,CHEBI:28498,acadesine,has_primary_input
382,GO:1903425,fluoride transmembrane transporter activity,CHEBI:17051,fluoride,has_primary_input


## Adjacency matrix

In [5]:
from subpred.adjacency_matrices import get_chebi_adjacency_matrix, get_go_adjacency_matrix

### GO adjacency

In [6]:
df_adj_matrix_go = get_go_adjacency_matrix(df_uniprot_goa=df_uniprot_goa, edges_filter={"is_a"})
df_adj_matrix_go

,GO:0000006,GO:0000007,GO:0000064,GO:0000095,GO:0000099,GO:0000100,GO:0000102,GO:0000295,GO:0000297,GO:0000319,...,GO:1901474,GO:1901505,GO:1901618,GO:1901680,GO:1901682,GO:1901702,GO:1902557,GO:1903089,GO:1903425,GO:1904680
GO:0000006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000064,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
GO:0000099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:1901702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:1902557,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
GO:1903089,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
GO:1903425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### ChEBI adjacency

In [7]:
df_adj_matrix_chebi = get_chebi_adjacency_matrix(
    df_go_chebi=df_go_chebi,
    edges_filter={"is_a"},
    primary_substrate_only=True,
)
df_adj_matrix_chebi

,CHEBI:13389,CHEBI:15354,CHEBI:15361,CHEBI:15377,CHEBI:15595,CHEBI:15676,CHEBI:15792,CHEBI:15927,CHEBI:16189,CHEBI:16199,...,CHEBI:61109,CHEBI:61292,CHEBI:61293,CHEBI:61336,CHEBI:63063,CHEBI:63299,CHEBI:64709,CHEBI:68452,CHEBI:77847,CHEBI:83821
CHEBI:13389,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:15354,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:15361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:15377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:15595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEBI:63299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:64709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:68452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:77847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Chemical similarity

Here, we are using the morgan fingerprints. There are other options that might be better:


In [16]:
from subpred.chemical_similarity import get_pairwise_similarity, tanimoto_chebi_to_go

dict_tanimoto_matrices_chebi = {
    f"tanimoto_{method}": get_pairwise_similarity(
        df_go_chebi=df_go_chebi, fingerprint_method=method
    )
    for method in ["morgan", "atompairs", "torsions", "maccs"]
}

[14:08:39] WARNING: not removing hydrogen atom without neighbors
[14:08:43] WARNING: not removing hydrogen atom without neighbors
[14:08:46] WARNING: not removing hydrogen atom without neighbors
[14:08:50] WARNING: not removing hydrogen atom without neighbors


In [18]:
dict_tanimoto_matrices_go = {
    f"{matrix_name}_go_{agg_method}": tanimoto_chebi_to_go(
        df_tanimoto_chebi=df_tanimoto_chebi,
        df_go_chebi=df_go_chebi,
        agg_function=agg_method,
        primary_input_only=True,
    )
    for agg_method in ["mean", "median", "min", "max"]
    for matrix_name, df_tanimoto_chebi in dict_tanimoto_matrices_chebi.items()
}
dict_tanimoto_matrices_go

{'tanimoto_morgan_mean': go_id2      GO:0000064  GO:0000095  GO:0000100  GO:0000102  GO:0000297  \
 go_id1                                                                   
 GO:0000064    1.000000    0.218750    0.500000    0.500000    0.222222   
 GO:0000095    0.218750    1.000000    0.311475    0.230769    0.028986   
 GO:0000100    0.500000    0.311475    1.000000    0.517241    0.060606   
 GO:0000102    0.500000    0.230769    0.517241    1.000000    0.060606   
 GO:0000297    0.222222    0.028986    0.060606    0.060606    1.000000   
 ...                ...         ...         ...         ...         ...   
 GO:1901235    0.212121    0.130435    0.235294    0.235294    0.031250   
 GO:1901239    0.250000    0.096774    0.230769    0.230769    0.045455   
 GO:1902557    0.057143    0.493333    0.055556    0.055556    0.015152   
 GO:1903089    0.074074    0.352113    0.071429    0.071429    0.020000   
 GO:1903425    0.000000    0.000000    0.000000    0.000000    0.000000   
 

## Overlap Matrix

### GO

In [9]:
from subpred.overlap_matrix import get_go_overlap_matrix, plot_go_overlap_matrix

df_go_overlaps = get_go_overlap_matrix(df_uniprot_goa=df_uniprot_goa, exclude_iea=EXCLUDE_IEA_GO_TERMS)
df_go_overlaps

go_id2,GO:0000006,GO:0000007,GO:0000064,GO:0000095,GO:0000099,GO:0000100,GO:0000102,GO:0000295,GO:0000297,GO:0000319,...,GO:1901474,GO:1901505,GO:1901618,GO:1901680,GO:1901682,GO:1901702,GO:1902557,GO:1903089,GO:1903425,GO:1904680
go_id1,,,,,,,,,,,,,,,,,,,,,
GO:0000006,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
GO:0000007,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
GO:0000064,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GO:0000095,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
GO:0000099,0,0,0,0,3,0,1,0,0,0,...,0,0,0,1,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:1901702,1,1,0,0,0,0,0,8,0,1,...,3,8,3,0,9,72,0,0,2,0
GO:1902557,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,1,0,0,0
GO:1903089,0,0,0,0,0,0,0,0,0,0,...,2,2,0,0,1,0,0,2,0,0


In [10]:
# plot_go_overlap_matrix(df_go_overlaps=df_go_overlaps, df_uniprot_goa=df_uniprot_goa)

### ChEBI

In [11]:
from subpred.overlap_matrix import get_chebi_overlaps

df_chebi_overlaps = get_chebi_overlaps(df_go_chebi=df_go_chebi, df_uniprot_goa=df_uniprot_goa, exclude_iea=EXCLUDE_IEA_GO_TERMS, primary_input_only=True)
df_chebi_overlaps

,CHEBI:13389,CHEBI:15354,CHEBI:15361,CHEBI:15377,CHEBI:15595,CHEBI:15676,CHEBI:15792,CHEBI:15927,CHEBI:16189,CHEBI:16199,...,CHEBI:61109,CHEBI:61292,CHEBI:61293,CHEBI:61336,CHEBI:63063,CHEBI:63299,CHEBI:64709,CHEBI:68452,CHEBI:77847,CHEBI:83821
CHEBI:13389,2,0,1,0,0,0,0,0,0,0,...,0,0,2,0,0,0,1,0,0,0
CHEBI:15354,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
CHEBI:15361,1,0,4,0,0,0,0,0,0,0,...,0,0,1,0,0,0,4,0,0,0
CHEBI:15377,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHEBI:15595,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEBI:63299,0,0,0,0,0,0,0,2,0,0,...,2,0,10,2,0,25,3,2,1,0
CHEBI:64709,1,0,4,0,1,0,1,0,2,0,...,0,0,3,6,2,3,65,3,0,5
CHEBI:68452,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,2,3,8,0,2
CHEBI:77847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


## GO Semantic Similarity Matrix


In [12]:
from subpred.go_semantic_similarity import get_go_semantic_similarities

df_semantic_similarity_wang = get_go_semantic_similarities(  # TODO tcss cutoff?
    aspect="F",
    organism_ids=ORGANISM_IDS,
    go_id_subset=df_uniprot_goa.go_id_ancestor.unique(),  # Transporters
    method="Wang",  # "Resnik", "Lin", "Rel", "Jiang", "TCSS" and "Wang"
    conversion_method="internal",
    parallelization_method="multicore"
)
df_semantic_similarity_wang

R[write to console]: Error in (function ()  : 
  org.Sc.sgdPFAM is defunct. Please use select() if you need access to
  PFAM or PROSITE accessions.

/home/andy/mambaforge/envs/subpred4/lib/python3.10/site-packages/rpy2/robjects/packages.py:263: UserWarning: R C-API Rf_findVarInFrame()
  warn(str(rre))
R[write to console]: preparing gene to GO mapping data...



,GO:0000006,GO:0000007,GO:0000064,GO:0000095,GO:0000099,GO:0000100,GO:0000102,GO:0000295,GO:0000297,GO:0000319,...,GO:1901474,GO:1901505,GO:1901618,GO:1901680,GO:1901682,GO:1901702,GO:1902557,GO:1903089,GO:1903425,GO:1904680
GO:0000006,1.000,0.713,0.300,0.330,0.233,0.267,0.166,0.114,0.326,0.267,...,0.252,0.252,0.377,0.205,0.351,0.349,0.191,0.211,0.293,0.260
GO:0000007,0.713,1.000,0.300,0.330,0.233,0.267,0.166,0.114,0.326,0.267,...,0.252,0.252,0.377,0.205,0.351,0.349,0.191,0.211,0.293,0.260
GO:0000064,0.300,0.300,1.000,0.471,0.565,0.381,0.538,0.247,0.473,0.381,...,0.404,0.404,0.488,0.450,0.310,0.518,0.394,0.326,0.408,0.395
GO:0000095,0.330,0.330,0.471,1.000,0.520,0.588,0.366,0.265,0.495,0.588,...,0.446,0.446,0.564,0.423,0.609,0.548,0.439,0.350,0.475,0.431
GO:0000099,0.233,0.233,0.565,0.520,1.000,0.548,0.543,0.217,0.321,0.548,...,0.396,0.396,0.521,0.755,0.616,0.364,0.408,0.310,0.430,0.387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:1901702,0.349,0.349,0.518,0.548,0.364,0.434,0.311,0.363,0.653,0.434,...,0.590,0.590,0.553,0.318,0.284,1.000,0.311,0.445,0.475,0.553
GO:1902557,0.191,0.191,0.394,0.439,0.408,0.457,0.320,0.636,0.281,0.457,...,0.332,0.472,0.433,0.344,0.451,0.311,1.000,0.369,0.362,0.331
GO:1903089,0.211,0.211,0.326,0.350,0.310,0.368,0.208,0.301,0.398,0.368,...,0.698,0.698,0.465,0.278,0.276,0.445,0.369,1.000,0.398,0.621
GO:1903425,0.293,0.293,0.408,0.475,0.430,0.503,0.258,0.308,0.419,0.673,...,0.525,0.525,0.651,0.358,0.460,0.475,0.362,0.398,1.000,0.497


## Pairwise ML models

- find label combinations with enough samples

In [13]:
from subpred.go_prediction import (
    get_model_evaluation_matrix_parallel,
    process_pairwise_eval_results,
    plot_results_as_heatmap,
)

pairwise_eval_results = get_model_evaluation_matrix_parallel(
    df_sequences,
    df_uniprot_goa,
    exclude_iea=EXCLUDE_IEA_GO_TERMS,
    standardize_samples=True,
    multi_output=True,
    min_samples_per_class=20,
    min_unique_samples_per_class=20,
    model_name="svc_multi",
    n_jobs=-1,
)

df_train_results, df_test_results = process_pairwise_eval_results(
    pairwise_eval_results=pairwise_eval_results, df_uniprot_goa=df_uniprot_goa, convert_go_ids_to_terms=False
)
# TODO graph with sample stats: y axis number of models, x axis samples per class, multi vs single
# TODO feature selection/pca on tera improved scores by quite a bit
# TODO with feature combinator
# TODO pca fewer steps

In [14]:
SHOW_PLOTS=0
if SHOW_PLOTS:
    plot_results_as_heatmap(
        df_train_results, title="average grid search test scores for each label across CV folds"
    )
    plot_results_as_heatmap(
        df_train_results,
        title="average grid search test scores for each label across CV folds, sorted",
        sort=True,
    )
    plot_results_as_heatmap(
        df_test_results, title="average independent test set scores for each label across CV folds"
    )
    plot_results_as_heatmap(
        df_test_results,
        title="average independent test set scores for each label across CV folds, sorted",
        sort=True,
    )

## Pairwise sequence sim matrix

In [6]:
from subpred.sequence_identity import (
    get_pairwise_alignment_scores,
    get_aggregated_sequence_alignments_go,
)

df_protein_identity, df_protein_alignment_scores = get_pairwise_alignment_scores(
    df_sequences=df_sequences, df_uniprot_goa=df_uniprot_goa, exclude_iea=EXCLUDE_IEA_GO_TERMS
)

In [24]:
dict_pairwise_alignment_scores = dict()
for aggr_method in [
    "median",
    "mean",
    "max",
    "min",
]:
    for score_name, score_matrix in zip(["identity", "alignment_score"], [df_protein_identity, df_protein_alignment_scores]):
        dict_pairwise_alignment_scores[
            f"go_{aggr_method}_sequence_{score_name}"
        ] = get_aggregated_sequence_alignments_go(
            df_uniprot_goa=df_uniprot_goa,
            df_protein_scores=score_matrix,
            exclude_iea=EXCLUDE_IEA_GO_TERMS,
            aggr_method=aggr_method,
        )

In [ ]:
dict_pairwise_alignment_scores

## Comparisons

### Datasets

- GO terms that are directly related vs. not
- GO terms that have a common parent vs. not
- GO terms that have a common ancestor vs. not (need to set depth limit, otherwise they all do)
- GO terms that a tanimoto coefficient above 0.5 (or other threshold) vs. below
- GO terms with max/median sequence identity

### Comparisons

- SVM scores
- Sequence identities (max, median)
- Overlaps in protein annotation
- Overlaps in chebi annotation

### Other ideas:

- Compare different ways to divide the transporter GO tree
- One of the scores on y axis, the rest in violin plot
    - Violin plots or similar

In [16]:
# Dataframes: for parameters:
# ORGANISM_IDS={559292}
# SWISSPROT_ONLY=False
# MAX_SEQUENCE_EVIDENCE_CODE = 1
# EXCLUDE_IEA_GO_TERMS=False

# TODO ChemSim: which fingerprint for similarity method?
# TODO ChemSim: filter other chebi dfs for those with valid fingerprint? Should remove abstract terms.
# TODO SemSim: which semantic similarity method? wang is fastest, tcss also fast
# TODO SeqSim: measures other than identity and median?
# TODO ML: min samples per class? Min unique samples?
# TODO ML: Feature selection and/or PCA?
# TODO ML: analysis with and without IEA terms?
# TODO compare scores inside of df to scores in entire df. for example, the max median sequence similatity here is 14, but in the overall df it is 100.

df_sequences                            #  332 x   5
df_uniprot_goa                          # 7737 x   8, 211 direct annot, 288 with ancestors
df_go_chebi                             #  384 x   5, primary substrate chebi terms: 159
        
df_adj_matrix_go                        #  288 x 288
df_go_overlaps                          #  288 x 288
df_semantic_similarity_wang             #  288 x 288

dict_pairwise_alignment_scores          #  288 x 288

dict_tanimoto_matrices_go               #  131 x 131
         
df_adj_matrix_chebi                     #  159 x 159
df_chebi_overlaps                       #  159 x 159
dict_tanimoto_matrices_chebi            #   89 x  89
         
df_train_results                        #   36 x  36, sparse, asymetrical
df_test_results                         #   36 x  36, sparse, asymetrical
pass

# TODO more matrices, generated with different parameters. max, min, mean, needleman-wunsch score, evaluation with PCA, common parent,...

In [22]:
print("unique proteins:", df_sequences.index.unique().shape[0])
print("unique directly annotated go terms:", df_uniprot_goa.go_id.unique().shape[0])
print("unique annotated go terms:", df_uniprot_goa.go_id_ancestor.unique().shape[0])
print("GO terms annotated with ChEBI terms", df_go_chebi.go_id.unique().shape[0])
print("Unique ChEBI terms", df_go_chebi.chebi_id.unique().shape[0])
print("Unique ChEBI terms usable for Tanimoto:", dict_tanimoto_matrices_chebi["tanimoto_atompairs"].index.unique().shape[0])

unique proteins: 332
unique directly annotated go terms: 211
unique annotated go terms: 288
GO terms annotated with ChEBI terms 226
Unique ChEBI terms 181
Unique ChEBI terms usable for Tanimoto: 89


### What determines good classification performance?

In [17]:
df_test_results_sparse = df_test_results.unstack().reset_index(name="test_score")
df_test_results_sparse = df_test_results_sparse[
    ~df_test_results_sparse.test_score.isnull()
].reset_index(drop=True)
df_test_results_sparse

,neg_label,pos_label,test_score
0,GO:0005342,GO:0005351,0.942857
1,GO:0005342,GO:0005402,0.942857
2,GO:0005342,GO:0008320,0.843333
3,GO:0005342,GO:0008324,0.927823
4,GO:0005342,GO:0015075,0.928203
...,...,...,...
803,GO:1901702,GO:0022884,0.560952
804,GO:1901702,GO:0022890,0.919998
805,GO:1901702,GO:0042626,0.882222
806,GO:1901702,GO:0046873,0.858109


In [14]:
df_test_results_sparse["is_a"] = df_test_results_sparse.apply(
    lambda row: int(df_adj_matrix_go.at[row.neg_label, row.pos_label]), axis=1
)  # TODO has same parent?
df_test_results_sparse["overlap"] = df_test_results_sparse.apply(
    lambda row: df_go_overlaps.at[row.neg_label, row.pos_label], axis=1
)
df_test_results_sparse["train_score"] = df_test_results_sparse.apply(
    lambda row: df_train_results.at[row.neg_label, row.pos_label], axis=1
)
df_test_results_sparse["semantic_sim"] = df_test_results_sparse.apply(
    lambda row: df_semantic_similarity_wang.at[row.neg_label, row.pos_label], axis=1
)

for matrix_name, alignment_score_matrix in dict_pairwise_alignment_scores.items():
    df_test_results_sparse[matrix_name] = df_test_results_sparse.apply(
        lambda row: alignment_score_matrix.at[row.neg_label, row.pos_label], axis=1
    )

# TODO very similar scores?
# df_test_results_sparse["median_tanimoto_score"] = df_test_results_sparse.apply(
#     lambda row: df_tanimoto_go.at[row.neg_label, row.pos_label], axis=1
# )



NameError: name 'df_test_results_sparse' is not defined

In [23]:
for name, df_tanimoto_go in dict_tanimoto_matrices_go.items():
    df_test_results_sparse = df_test_results_sparse.merge(
        df_tanimoto_go.unstack().reset_index(name=name),
        how="left",
        left_on=["neg_label", "pos_label"],
        right_on=["go_id2", "go_id1"],
    ).drop(["go_id2",	"go_id1"], axis=1)

NameError: name 'df_test_results_sparse' is not defined

In [19]:
df_test_results_sparse.drop(["neg_label", "pos_label"],axis=1).corr()

,test_score,is_a,overlap,train_score,semantic_sim,median_sequence_ident,mean_maccs_tanimoto
test_score,1.000000,NaN,-0.032846,-0.057429,-0.179951,-0.043350,0.196933
is_a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
overlap,-0.032846,NaN,1.000000,-0.019540,0.190546,-0.052002,0.707107
train_score,-0.057429,NaN,-0.019540,1.000000,-0.190696,-0.038138,0.098862
semantic_sim,-0.179951,NaN,0.190546,-0.190696,1.000000,0.241144,0.670102
median_sequence_ident,-0.043350,NaN,-0.052002,-0.038138,0.241144,1.000000,-0.168560
mean_maccs_tanimoto,0.196933,NaN,0.707107,0.098862,0.670102,-0.168560,1.000000
